## 1.환경설정

### 1-1.필요한 라이브러리 다운로드

In [1]:
!pip install -Uq langchain langchain-core langchain-community langchain-upstage \
redis pandas python-dotenv tqdm datasets huggingface-hub


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import os
import json
import re
from typing import List, Dict, Any

from huggingface_hub import (
    HfApi, create_repo, repo_exists, upload_folder,
    list_repo_files, hf_hub_download
)
from datasets import load_dataset

import pandas as pd
from dotenv import load_dotenv
from tqdm import tqdm

# 1) Redis를 LangChain Community 모듈로 불러오기
from langchain_community.chat_message_histories import RedisChatMessageHistory

# 2) 테스트 LLM을 LangChain으로 불러오기(Upstage ChatUpstage)
from langchain_upstage import ChatUpstage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

c:\Users\user\.llmenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 1-2.환경설정 변수 설정

- .env file 예시

In [ ]:
# # Redis
# REDIS_URL=redis://localhost:6379
# KEY_PREFIX=message_store:        # LangChain 기본 prefix (바꿨으면 맞게)
# SESSION_IDS=sesac,devtest        # 콤마로 여러 세션 지정 (자동탐색 안 쓰는 간단 방식)

# # Local export
# OUTPUT_DIR=llm_sessions

# # Upstage
# UPSTAGE_API_KEY=upstage_xxx
# SOLAR_MODEL=solar-pro         # 필요시 solar-pro-2 등

# HF_DATASET_REPO=your-username/llm-chat-sessions
# HF_TOKEN=hf_xxx
# HF_PRIVATE=true
# LOCAL_CSV_DIR=llm_sessions


In [31]:
# redis 에서 session_id를 추출해서 env 파일로 
import redis

r = redis.Redis(host='localhost', port=6379, db=0)
keys = r.keys(pattern="message_store:*")

# redis는 내부적으로 bytes로 저장한다. 바로 str처럼 split 안 됨. decoding 해야함.
session_ids = [key.decode().split(':')[1] for key in keys]

# env에 환경변수 등록할 때는 str만 된다.
os.environ['SESSION_IDS'] = ','.join(session_ids)

In [32]:
load_dotenv()

REDIS_URL = os.getenv("REDIS_URL", "redis://localhost:6379")
KEY_PREFIX = os.getenv("KEY_PREFIX", "message_store:")
OUTPUT_DIR = os.getenv("OUTPUT_DIR", "llm_sessions")
os.makedirs(OUTPUT_DIR, exist_ok=True)

UPSTAGE_API_KEY = os.getenv("UPSTAGE_API_KEY")  # ChatUpstage가 env로 읽음
SOLAR_MODEL = os.getenv("SOLAR_MODEL", "solar-pro2")

# 세션ID 목록: 콤마 구분
# env에 변수로 등록할 때는 str이라 여기서 split으로 다시 list로 바꿔서 뽑는다.
# if s.strip() -> s (=id)가 공백 제거했을 때 false가 아니라면 (= s가 공백이 아니라면) 포함해라
SESSION_IDS = [s.strip() for s in os.getenv("SESSION_IDS", "").split(',') if s.strip()]

HF_REPO   = os.getenv("HF_DATASET_REPO", "jaewook/llm-chat-sessions")          # 예: your-username/llm-chat-sessions
HF_TOKEN  = os.getenv("HF_TOKEN")
HF_PRIVATE = str(os.getenv("HF_PRIVATE", "true")).lower() == "true"
LOCAL_DIR = os.getenv("LOCAL_CSV_DIR", "llm_sessions")  # 세션별 CSV 폴더

## 2.Redis 에서 chat data 불러와서 csv file 로 저장하기

### 2-1. Redis 에서 session id 로 chat data 불러오기

In [33]:
def _messages_to_rows(history) -> List[Dict[str, Any]]:
    """LangChain BaseMessage -> 표준 dict 리스트(role, content, timestamp)"""
    rows = []
    for m in history.messages:
        role = getattr(m, "type", None) or m.__class__.__name__.replace("Message", "").lower()
        content = m.content
        timestamp = None
        # additional_kwargs에 timestamp를 넣어뒀다면 꺼냄
        try:
            timestamp = (m.additional_kwargs or {}).get("timestamp")
        except Exception:
            pass
        rows.append({"role": role, "content": content, "timestamp": timestamp})
    return rows

In [34]:
def _messages_to_text(rows: List[Dict[str, Any]]) -> str:
    lines = []
    for r in rows:
        role = (r.get("role") or "user").upper()
        content = r.get("content") or ""
        lines.append(f"{role}: {content}")
    return "\n".join(lines)

In [35]:
def _chunk_text(text: str, chunk_size: int = 4000) -> List[str]:
    chunks = []
    i = 0
    while i < len(text):
        chunks.append(text[i:i+chunk_size])
        i += chunk_size
    return chunks

In [36]:
historys = []
for sid in tqdm(SESSION_IDS, desc="Exporting sessions"):
        history = RedisChatMessageHistory(
            session_id=sid,
            url=REDIS_URL,
            key_prefix=KEY_PREFIX
        )
        historys.append(history)

Exporting sessions:   0%|          | 0/7 [00:00<?, ?it/s]

Exporting sessions: 100%|██████████| 7/7 [00:00<00:00, 105.17it/s]


In [40]:
for i,mes in enumerate(historys[3].messages):
    print(i,mes)

0 content='안녕' additional_kwargs={} response_metadata={}
1 content='안녕하세용🍀' additional_kwargs={} response_metadata={'finish_reason': 'stop', 'model_name': 'solar-mini-250422'} id='run--5ffe59f4-776a-40a4-934e-178c8845e0c6-0'
2 content='현재 시간은?' additional_kwargs={} response_metadata={}
3 content='지금 시간은 비밀🍀 (농담이야~ 지금 당장 알려줄게! 뿅!)' additional_kwargs={} response_metadata={'finish_reason': 'stop', 'model_name': 'solar-mini-250422'} id='run--dd76b7d0-1a7b-4bfc-822d-02c9db3293e9-0'
4 content='알려줘' additional_kwargs={} response_metadata={}
5 content='음... 지금은 "너랑 놀 시간!" 뿅🍀' additional_kwargs={} response_metadata={'finish_reason': 'stop', 'model_name': 'solar-mini-250422'} id='run--8e05e8d5-4ea1-4409-a4ec-e287805c92b4-0'
6 content='attention mask가 뭐야?' additional_kwargs={} response_metadata={}
7 content='Attention mask는 머신 러닝에서 사용하는 마법의 마스크야! 특정 부분에 집중하게 도와줘 👀🍀' additional_kwargs={} response_metadata={'finish_reason': 'stop', 'model_name': 'solar-mini-250422'} id='run--e732a296-4e6d-41e0-bcff

### 2-2. 나눴던 대화 csv file 로 저장하기

In [41]:
if __name__ == "__main__":
    if not SESSION_IDS:
            raise SystemExit(
                "SESSION_IDS(.env)을 지정하세요. 예) SESSION_IDS=sesac,project42"
            )

    session_csv_paths = {}

    # Redis(LangChain Community)로 세션별 대화 불러와 CSV 저장
    for sid in tqdm(SESSION_IDS, desc="Exporting sessions"):
        history = RedisChatMessageHistory(
            session_id=sid,
            url=REDIS_URL,
            key_prefix=KEY_PREFIX
        )
        rows = _messages_to_rows(history)
        df = pd.DataFrame(rows)
        csv_path = os.path.join(OUTPUT_DIR, f"{sid}.csv")
        df.to_csv(csv_path, index=False, encoding="utf-8-sig")
        session_csv_paths[sid] = csv_path

    print(f"[OK] {len(session_csv_paths)}개 세션 CSV 저장 -> {OUTPUT_DIR}/")

Exporting sessions: 100%|██████████| 7/7 [00:00<00:00, 29.77it/s]

[OK] 7개 세션 CSV 저장 -> llm_sessions/


## 3. 이전에 나눈 대화 요약해서 저장하기

In [42]:
def get_llm(temperature: float = 0.0) -> ChatUpstage:
    # UPSTAGE_API_KEY 필요. 없으면 오류 발생.
    return ChatUpstage(model=SOLAR_MODEL, temperature=temperature)

SUMMARIZE_SYSTEM = (
    "너는 전문 요약가다. 다음 대화를 400~600단어 내로 요약하되, "
    "1) 의도/맥락 2) 핵심 포인트 3) 결정사항 4) To-Do 5) 열린 이슈 를 구조화해라. 한국어로."
)

def summarize_text(text: str) -> str:
    """긴 대화 텍스트를 map→reduce로 요약"""
    llm_map = get_llm(0.0)
    llm_reduce = get_llm(0.0)

    map_prompt = ChatPromptTemplate.from_messages([
        ("system", SUMMARIZE_SYSTEM),
        ("human", "다음 내용을 요약하라:\n\n{chunk}")
    ])
    map_chain = map_prompt | llm_map | StrOutputParser()

    reduce_prompt = ChatPromptTemplate.from_messages([
        ("system", SUMMARIZE_SYSTEM),
        ("human", "다음 부분 요약들을 하나로 통합하라:\n\n{parts}")
    ])
    reduce_chain = reduce_prompt | llm_reduce | StrOutputParser()

    chunks = _chunk_text(text, chunk_size=3500)
    parts = [map_chain.invoke({"chunk": c}) for c in chunks] if chunks else [""]
    merged = reduce_chain.invoke({"parts": "\n\n---\n\n".join(parts)})
    return merged.strip()

In [43]:
def ask_with_summary(summary_doc: str, query: str) -> str:
    llm = get_llm(0.2)
    prompt = ChatPromptTemplate.from_messages([
        ("system", "너는 유능한 한국어 비서다. 제공된 '이전 대화 요약'을 참고하여 정확하고 간결하게 답해라."),
        ("human", "[이전 대화 요약]\n{summary}\n\n[질문]\n{query}")
    ])
    chain = prompt | llm | StrOutputParser()
    return chain.invoke({"summary": summary_doc, "query": query}).strip()

In [44]:
# 테스트 LLM(ChatUpstage)으로 요약 + 질의 데모
# 세션을 하나 골라 요약 후, 질문 예시 실행
first_sid = SESSION_IDS[0]
df = pd.read_csv(session_csv_paths[first_sid])
text = _messages_to_text(df.to_dict(orient="records"))

summary = summarize_text(text)
print("\n[요약문 미리보기]\n", summary[:1000], "...\n")

answer = ask_with_summary(summary, query="이 대화의 다음 액션 아이템 3가지만 제안해줘.")
print("[응답]\n", answer)


[요약문 미리보기]
 ### **통합 요약: 업스테이지 및 AI 관련 정보 교환**  

#### **1. 의도/맥락**  
사용자는 AI 모델 "솔라"와의 대화를 통해 **엔비디아 회장 이름**, **업스테이지의 대표 및 연혁**, **기술 성과와 주요 사업**에 대한 정보를 요청했다. AI는 캐주얼한 어조(이모지 포함)로 답변하며, 업스테이지의 기술력과 솔라 모델의 역할을 강조했다. 이 대화는 업스테이지에 대한 기본 이해를 목적으로 진행되었으나, 기술적 세부사항과 공식 데이터 부재로 인해 추가 검증이 필요한 상황이다.  

#### **2. 핵심 포인트**  
- **엔비디아 회장**: 젠슨 황(Jensen Huang).  
- **업스테이지 대표**: 김성훈.  
- **업스테이지 연혁**:  
  - 창립 후 AI 모델 "솔라" 개발 → 현재 사용자와의 상호작용 단계.  
- **업스테이지의 성과**:  
  - 글로벌 AI 모델 "솔라" 출시 및 대중화.  
  - 기업용 AI 솔루션(예: 솔라) 제공.  
- **주요 사업**:  
  - 기업 대상 AI 기술 개발 및 솔루션 공급.  
  - 사용자 친화적인 AI 모델(솔라)을 통한 서비스 확장.  

#### **3. 결정사항**  
- 사용자는 업스테이지와 솔라에 대한 기본 정보를 확인했으며, AI의 답변에서 **기술력과 솔라 모델의 역할**을 인지했다.  
- AI는 업스테이지의 성과를 강조하며, 사용자와의 지속적 상호작용 가능성을 암시했다.  

#### **4. To-Do**  
- **사용자**:  
  - 업스테이지 공식 채널(웹사이트, SNS)에서 재정 현황, 파트너십, 정량적 성과(사용자 수, 시장 점유율) 등 추가 정보 확인.  
  - 솔라 모델의 구체적 기능(언어 처리, 멀티모달 능력) 및 기술적 세부사항(모델 아키텍처, 학습 데이터) 조사.  
- **AI/업스테이지**:  
  - 사용자 피드백을 반영해 솔라 모델의 정확성 및 전문성 개선.  
  - 기업용 솔루션의 사례

## 4. csv file HFdataset 에 업로드하기

In [ ]:
def upload_csv_folder_to_hf(LOCAL_DIR: str, repo_id: str, private: bool, token: str):
    """CSV 폴더를 HF dataset 레포의 /csv/ 경로에 업로드 (레포 없으면 생성)"""
    if not repo_id:
        raise ValueError("HF_DATASET_REPO(.env)을 지정하세요. e.g. your-username/llm-chat-sessions")
    if not os.path.isdir(LOCAL_DIR):
        raise FileNotFoundError(f"로컬 폴더가 없습니다: {LOCAL_DIR}")

    api = HfApi(token=token)

    # 데이터셋 레포 없으면 생성
    if not repo_exists(repo_id, repo_type="dataset", token=token):
        create_repo(repo_id, repo_type="dataset", private=private, token=token)
        print(f"[HF] 레포 생성: https://huggingface.co/datasets/{repo_id}")
    else:
        print(f"[HF] 레포 존재: https://huggingface.co/datasets/{repo_id}")

    # 폴더 업로드 → 레포 내 경로는 /csv
    upload_folder(
        repo_id=repo_id,
        repo_type="dataset",
        folder_path=LOCAL_DIR,
        path_in_repo="csv",
        token=token,
    )
    print(f"[HF] 업로드 완료: {LOCAL_DIR} → {repo_id}/csv/")

In [ ]:
# 1) 업로드
upload_csv_folder_to_hf(LOCAL_DIR, HF_REPO, HF_PRIVATE, HF_TOKEN)

## 5. 저장했던 HF dataset 다시 불러오기

In [ ]:
def load_csv_dataset_from_hf(repo_id: str, token: str):
    """
    HF 데이터셋 레포의 /csv/*.csv 파일들을 모두 받아서 하나의 split('all')로 로드.
    필요하면 세션별로 split을 나누는 로직도 추가 가능.
    """
    files = list_repo_files(repo_id=repo_id, repo_type="dataset", token=token)
    csv_files = [f for f in files if f.startswith("csv/") and f.lower().endswith(".csv")]
    if not csv_files:
        raise RuntimeError("레포에 csv/*.csv 파일이 없습니다. 먼저 업로드하세요.")

    # 원격 csv들을 로컬로 내려받아 경로 리스트 구성
    local_paths = [hf_hub_download(repo_id=repo_id, repo_type="dataset", filename=f, token=token)
                   for f in csv_files]

    # huggingface datasets로 통합 로드
    ds = load_dataset("csv", data_files={"all": local_paths})
    print(f"[HF] 로드 완료: splits={list(ds.keys())}, rows={len(ds['all'])}")
    return ds

In [ ]:
# 2) 불러오기
ds = load_csv_dataset_from_hf(HF_REPO, HF_TOKEN)

In [ ]:
df = ds["all"].to_pandas() # 판다스로 변환
print(df.head(3))